In [33]:
import os
import pandas as pd
import random
from copy import deepcopy
from copy import deepcopy

random.seed(42)

def reorder_dataframe_columns(df, preferred_columns):
    # Ensure all preferred columns are in the DataFrame
    valid_preferred_columns = [col for col in preferred_columns if col in df.columns]
    # Get the remaining columns not in preferred_columns, maintaining original order
    remaining_columns = [col for col in df.columns if col not in valid_preferred_columns]
    # Combine the lists
    new_column_order = valid_preferred_columns + remaining_columns
    # Reorder the DataFrame and return
    return df[new_column_order]




folder_path = 'data'
datasets = {}

for filename in os.listdir(folder_path):
    if filename.endswith('.csv'):
        file_path = os.path.join(folder_path, filename)
        df = pd.read_csv(file_path)
        datasets[os.path.splitext(filename)[0]] = df
for el in datasets:
    print(el)
print(len(datasets))

/tmp/ipykernel_44206/3703000730.py:28: DtypeWarning: Columns (17,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


DIAGNOSES_ICD
DRGCODES
LABEVENTS
D_ITEMS
D_LABITEMS
PROCEDUREEVENTS_MV
INPUTEVENTS_MV
INPUTEVENTS_CV
PATIENTS
ICUSTAYS
ADMISSIONS
MICROBIOLOGYEVENTS
CHARTEVENTS
SERVICES
D_ICD_PROCEDURES
CPTEVENTS
TRANSFERS
OUTPUTEVENTS
PRESCRIPTIONS
D_ICD_DIAGNOSES
DATETIMEEVENTS
CAREGIVERS
D_CPT
CALLOUT
NOTEEVENTS
PROCEDURES_ICD
26


/tmp/ipykernel_44206/3703000730.py:28: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [34]:
datasets_with_subject_id = []
for name, df in datasets.items():
    if 'subject_id' in df.columns:
        datasets_with_subject_id.append(name)
        
other_datasets = set(datasets) - set(datasets_with_subject_id)

print(f"Dataset that contain the column 'subject_id' ({len(datasets_with_subject_id)}): {datasets_with_subject_id}")
print(f"Other ({len(other_datasets)}): {other_datasets}")

Dataset that contain the column 'subject_id' (20): ['DIAGNOSES_ICD', 'DRGCODES', 'LABEVENTS', 'PROCEDUREEVENTS_MV', 'INPUTEVENTS_MV', 'INPUTEVENTS_CV', 'PATIENTS', 'ICUSTAYS', 'ADMISSIONS', 'MICROBIOLOGYEVENTS', 'CHARTEVENTS', 'SERVICES', 'CPTEVENTS', 'TRANSFERS', 'OUTPUTEVENTS', 'PRESCRIPTIONS', 'DATETIMEEVENTS', 'CALLOUT', 'NOTEEVENTS', 'PROCEDURES_ICD']
Other (6): {'D_LABITEMS', 'D_CPT', 'D_ICD_DIAGNOSES', 'CAREGIVERS', 'D_ITEMS', 'D_ICD_PROCEDURES'}


In [35]:
df = datasets['PATIENTS']
patient_id = random.choice(df['subject_id'].tolist())

print(f"random `subject_id`: {patient_id}")

random `subject_id`: 42412


In [36]:
datasets_patient = {}

for name, df in datasets.items():
    if 'subject_id' in df.columns:
        # Filtrare le righe dove 'subject_id' è uguale a target_subject_id
        filtered_df = df[df['subject_id'] == patient_id]
        print(name, filtered_df.shape, end=' ')
        # Se ci sono righe filtrate, aggiungerle al dizionario dei dataset filtrati
        if not filtered_df.empty:
            datasets_patient[name] = filtered_df
            print()
        else: 
            print('*EMPTY*')

DIAGNOSES_ICD (21, 5) 
DRGCODES (3, 8) 
LABEVENTS (950, 9) 
PROCEDUREEVENTS_MV (5, 25) 
INPUTEVENTS_MV (12, 31) 
INPUTEVENTS_CV (0, 22) *EMPTY*
PATIENTS (1, 8) 
ICUSTAYS (1, 12) 
ADMISSIONS (1, 19) 
MICROBIOLOGYEVENTS (3, 16) 
CHARTEVENTS (1573, 15) 
SERVICES (1, 6) 
CPTEVENTS (8, 12) 
TRANSFERS (3, 13) 
OUTPUTEVENTS (20, 13) 
PRESCRIPTIONS (45, 19) 
DATETIMEEVENTS (19, 14) 
CALLOUT (0, 24) *EMPTY*
NOTEEVENTS (0, 11) *EMPTY*
PROCEDURES_ICD (2, 5) 


In [39]:
filtered_datasets.keys()

dict_keys(['D_ITEMS', 'D_LABITEMS', 'D_ICD_PROCEDURES', 'D_ICD_DIAGNOSES', 'CAREGIVERS', 'D_CPT'])

In [42]:
filtered_datasets = {
    key: value for key, value in datasets.items() if key not in datasets_with_subject_id
}

merged_datasets = dict()


for el in datasets_patient:
    print(el)
    m = deepcopy(datasets_patient[el])
    if "cgid" in m.columns:
        m = pd.merge(m, 
                     filtered_datasets["CAREGIVERS"].add_suffix('_CAREGIVERS').rename(columns={'cgid_CAREGIVERS':'cgid'}), 
                     on="cgid", 
                     how="left",)
    if "icd9_code" in m.columns:
        col_type = m["icd9_code"].dtype
        if el == "PROCEDURES_ICD":
            m = pd.merge(
                m, 
                filtered_datasets["D_ICD_PROCEDURES"].add_suffix("_D_ICD_PROCEDURES").rename(columns={'icd9_code_D_ICD_PROCEDURES': 'icd9_code'}), 
                on="icd9_code", 
                how="left", 
            )
        else:
            m = pd.merge(
                m, 
                filtered_datasets["D_ICD_DIAGNOSES"].add_suffix("_D_ICD_DIAGNOSES").rename(columns={'icd9_code_D_ICD_DIAGNOSES': 'icd9_code'}), 
                on="icd9_code", 
                how="left",
            )
    if "itemid" in m.columns:
        if el == "LABEVENTS":
            m = pd.merge(m, 
                         filtered_datasets["D_LABITEMS"].add_suffix("_D_LABITEMS").rename(columns={'itemid_D_LABITEMS': 'itemid'}), 
                         on="itemid", 
                         how="left",
                        )
        else:
            m = pd.merge(m, 
                         filtered_datasets["D_ITEMS"].add_suffix("_D_ITEMS").rename(columns={'itemid_D_ITEMS': 'itemid'}), 
                         on="itemid", 
                         how="left",)
    merged_datasets[el] = deepcopy(reorder_dataframe_columns(m, datasets[el].columns))
    print(merged_datasets[el].shape)

DIAGNOSES_ICD
(21, 8)
DRGCODES
(3, 8)
LABEVENTS
(950, 14)
PROCEDUREEVENTS_MV
(5, 37)
INPUTEVENTS_MV
(12, 43)
PATIENTS
(1, 8)
ICUSTAYS
(1, 12)
ADMISSIONS
(1, 19)
MICROBIOLOGYEVENTS
(3, 16)
CHARTEVENTS
(1573, 27)
SERVICES
(1, 6)
CPTEVENTS
(8, 12)
TRANSFERS
(3, 13)
OUTPUTEVENTS
(20, 25)
PRESCRIPTIONS
(45, 19)
DATETIMEEVENTS
(19, 26)
PROCEDURES_ICD
(2, 8)


In [43]:
file_path = 'Merged_tables'
for el in merged_datasets:
    file_name = f'{file_path}/merged_{el}.xlsx'
    print(file_name)
    merged_datasets[el].to_excel(file_name, index=False)

Merged_tables/merged_DIAGNOSES_ICD.xlsx
Merged_tables/merged_DRGCODES.xlsx
Merged_tables/merged_LABEVENTS.xlsx
Merged_tables/merged_PROCEDUREEVENTS_MV.xlsx
Merged_tables/merged_INPUTEVENTS_MV.xlsx
Merged_tables/merged_PATIENTS.xlsx
Merged_tables/merged_ICUSTAYS.xlsx
Merged_tables/merged_ADMISSIONS.xlsx
Merged_tables/merged_MICROBIOLOGYEVENTS.xlsx
Merged_tables/merged_CHARTEVENTS.xlsx
Merged_tables/merged_SERVICES.xlsx
Merged_tables/merged_CPTEVENTS.xlsx
Merged_tables/merged_TRANSFERS.xlsx
Merged_tables/merged_OUTPUTEVENTS.xlsx
Merged_tables/merged_PRESCRIPTIONS.xlsx
Merged_tables/merged_DATETIMEEVENTS.xlsx
Merged_tables/merged_PROCEDURES_ICD.xlsx
